In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [44]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import datetime as dt
# import yfinance as yf
# #import pandas_ta
# import warnings
# warnings.filterwarnings('ignore')

# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
# sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
# symbols_list = sp500['Symbol'].unique().tolist()
# end_date = '2025-06-01'
# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)
# df = yf.download(tickers=symbols_list,
#                  start=start_date,
#                  end=end_date).stack()
df.columns =  df.columns.str.lower()
df

Price                   close        high         low        open       volume
Date       Ticker                                                             
2017-06-05 A        56.852486   57.266642   56.786597   57.228990    1473300.0
           AAPL     35.854847   35.975971   35.745373   35.950349  101326800.0
           ABBV     47.947414   48.195993   47.499968   47.762754    4918100.0
           ABT      40.446308   40.559094   40.238092   40.402930    7810600.0
           ACGL     31.167286   31.420855   31.094383   31.249697    2004900.0
...                       ...         ...         ...         ...          ...
2025-05-30 XYL     126.040001  126.690002  125.199997  126.519997    2624100.0
           YUM     143.940002  144.479996  143.089996  144.039993    3184400.0
           ZBH      92.169998   93.040001   91.440002   92.940002    3505200.0
           ZBRA    289.769989  290.559998  283.920013  289.929993     656100.0
           ZTS     168.630005  169.789993  165.339996  166.919998    5467800.0

[990176 rows x 5 columns]

In [ ]:
#Technical indicators for each stock
Garman-klass volatility
RSI
bollinger bands
ATR
MACD
Dollar Volume

In [48]:
data = [('m',float(3.28),'ft'),('ft',float(12),'in'),('hr',float(60),'min'),('min',float(60),'sec')]

In [80]:
from collections import defaultdict, deque

graph = defaultdict(list)
for from_unit,multiplier,to_unit in data:
    graph[from_unit].append((multiplier,to_unit))
    graph[to_unit].append((1/multiplier,from_unit))

def answer(curr_amt,curr_unit):
    queue = deque([(curr_amt,curr_unit)])
    if curr_unit not in graph or desired_unit not in graph:
        return 'Not convertible'
    visited = set()
    visited.add(curr_unit)
    while queue:
        curr_amt, curr_unit = queue.popleft()
        if curr_unit == desired_unit:
            return curr_amt
        for multiplier,next_unit in graph[curr_unit]:
            if next_unit not in visited:
                visited.add(next_unit)
                curr_amt = curr_amt *multiplier
                queue.append((curr_amt, next_unit))
    return 'Not convertible'

In [92]:
desired_unit = 'm'
answer(354.24,'in')

9.0

In [60]:
graph

defaultdict(list,
            {'m': [(3.28, 'ft')],
             'ft': [(0.3048780487804878, 'm'), (12.0, 'in')],
             'in': [(0.08333333333333333, 'ft')],
             'hr': [(60.0, 'min')],
             'min': [(0.016666666666666666, 'hr'), (60.0, 'sec')],
             'sec': [(0.016666666666666666, 'min')]})